In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurar caminhos (ajuste se o notebook não estiver na pasta 'src')
# Assumindo que o notebook está na raiz 'ML Avancado'
DB_PATH = Path() / ".." / "banco de dados" / "crash_bot_historico.db"

# Conectar ao banco de dados e carregar os dados
try:
    with sqlite3.connect(DB_PATH) as conn:
        # Carrega a tabela principal de multiplicadores
        query = """
            SELECT timestamp, multiplicador
            FROM multiplicadores_historico
            ORDER BY timestamp ASC
        """
        df = pd.read_sql_query(query, conn)
    
    # Converter 'timestamp' para o formato datetime do pandas
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Definir o timestamp como o índice (crucial para Time Series)
    df = df.set_index('timestamp').sort_index()

    print(f"Sucesso: {len(df)} rodadas carregadas do banco de dados.")
    print("Últimas 5 rodadabs:")
    display(df.tail())

except Exception as e:
    print(f"Erro ao carregar o banco de dados: {e}")

Sucesso: 5911 rodadas carregadas do banco de dados.
Últimas 5 rodadabs:


,multiplicador
timestamp,
2025-11-01 08:45:26.091620,1.59
2025-11-01 08:46:03.320203,8.69
2025-11-01 08:46:21.307709,1.29
2025-11-01 08:46:42.618357,1.60
2025-11-01 08:47:05.637626,1.91


In [2]:
# Célula de Análise 1: Engenharia de Features de Sequência (Streaks)
# Precisamos calcular as sequências de baixos e identificar os gatilhos.

# --- 1. Definição das Constantes da Análise ---
LOW_THRESHOLD = 1.99  # Limite de 'Baixo' (como você mencionou)
TARGET_MARTINGALE = 1.84 # Alvo do Martingale (como você mencionou)
MARTINGALE_TRIGGER = 8   # Gatilho de aposta (como você mencionou)
SUPER_STREAK = 12        # Limite da "fase arrecadatória" (como você mencionou)

# --- 2. Calcular 'low_streak' (Quantas rodadas consecutivas < 1.99x) ---
# Usamos .shift(1) para que a contagem em qualquer linha X
# represente a sequência que aconteceu ANTES da rodada X.
is_low = (df['multiplicador'].shift(1) < LOW_THRESHOLD)
cumulative_lows = is_low.astype(int).cumsum()
streak_breaks = cumulative_lows.where(~is_low)
filled_breaks = streak_breaks.ffill().fillna(0)
df['low_streak'] = (cumulative_lows - filled_breaks).astype(int)

# --- 3. Calcular 'streak_length' (O comprimento total de cada sequência) ---
# Esta é uma análise diferente: ela marca o *fim* de uma sequência
# com o seu comprimento total.
df['is_low'] = (df['multiplicador'] < LOW_THRESHOLD)
# Identifica o fim de uma sequência (quando muda de True para False)
df['streak_end'] = (df['is_low'] == True) & (df['is_low'].shift(-1) == False)
# Identifica o ID único de cada sequência
df['streak_id'] = df['is_low'].diff().astype(float).fillna(0.0).cumsum()
# Conta o tamanho de cada sequência
streak_lengths = df[df['is_low'] == True].groupby('streak_id').size()
# Mapeia o tamanho total de volta para a linha final da sequência
df['streak_length'] = df['streak_id'].map(streak_lengths)
# Preenche os NaNs
df['streak_length'] = df['streak_length'].ffill().fillna(0)
# Limpa as linhas onde a sequência não terminou
df.loc[~df['streak_end'], 'streak_length'] = 0 


print(f"Sucesso: Colunas 'low_streak' e 'streak_length' criadas.")
print("\nVisão geral das sequências longas encontradas:")
# Mostra as 5 sequências de baixos mais longas que o bot registrou
display(df[df['streak_length'] >= SUPER_STREAK][['multiplicador', 'streak_length']])

Sucesso: Colunas 'low_streak' e 'streak_length' criadas.

Visão geral das sequências longas encontradas:


,multiplicador,streak_length
timestamp,,
2025-10-25 00:27:01.334663,1.50,13.0
2025-10-27 13:05:18.474230,1.02,13.0


In [3]:
# Célula de Análise 2: Efetividade da Estratégia (1.84x após 8 Baixos)

print(f"Analisando a taxa de acerto para o alvo {TARGET_MARTINGALE}x "
      f"após {MARTINGALE_TRIGGER} multiplicadores < {LOW_THRESHOLD}x\n")

# --- 1. Encontrar todos os "Gatilhos" ---
# Encontra todas as linhas onde a sequência atingiu 8.
trigger_rows = df[df['low_streak'] == MARTINGALE_TRIGGER]
total_triggers = len(trigger_rows)

if total_triggers > 0:
    # --- 2. Analisar o Resultado (a rodada seguinte) ---
    # Pega o multiplicador real da rodada que foi apostada
    results = df.loc[trigger_rows.index, 'multiplicador']
    
    # --- 3. Calcular a Taxa de Acerto ---
    hits = (results >= TARGET_MARTINGALE).sum()
    misses = total_triggers - hits
    hit_rate = (hits / total_triggers) * 100
    
    print(f"--- Análise do Gatilho Padrão ({MARTINGALE_TRIGGER} baixos) ---")
    print(f"Total de vezes que a estratégia foi acionada: {total_triggers}")
    print(f"Total de Acertos (>= {TARGET_MARTINGALE}x): {hits}")
    print(f"Total de Erros (< {TARGET_MARTINGALE}x): {misses}")
    print(f"Taxa de Acerto (Hit Rate): {hit_rate:.2f}%")

    # --- 4. Análise Aprofundada (Como a "Fase Arrecadatória" afeta?) ---
    # O que acontece se o bot continuar apostando em sequências mais longas?
    print("\n--- Análise de Efetividade vs. Comprimento da Sequência ---")
    
    # Encontra todas as apostas que teriam ocorrido DEPOIS de 8 baixos
    all_martingale_bets = df[df['low_streak'] >= MARTINGALE_TRIGGER]
    
    if len(all_martingale_bets) > 0:
        # Agrupa por quantos baixos já ocorreram (8, 9, 10, 11, 12...)
        # E calcula a taxa de acerto para cada um
        streaks_df = all_martingale_bets.groupby('low_streak')['multiplicador'].agg(
            total_apostas='count',
            acertos=lambda x: (x >= TARGET_MARTINGALE).sum()
        ).reset_index()
        
        streaks_df['taxa_de_acerto_%'] = (streaks_df['acertos'] / streaks_df['total_apostas']) * 100
        
        print(f"Taxa de acerto do alvo {TARGET_MARTINGALE}x em cada etapa da sequência:")
        display(streaks_df)
    else:
        print("Nenhuma sequência atingiu 8 baixos para esta análise.")

else:
    print(f"Nenhuma sequência de {MARTINGALE_TRIGGER} baixos foi encontrada nos dados.")

Analisando a taxa de acerto para o alvo 1.84x após 8 multiplicadores < 1.99x

--- Análise do Gatilho Padrão (8 baixos) ---
Total de vezes que a estratégia foi acionada: 16
Total de Acertos (>= 1.84x): 10
Total de Erros (< 1.84x): 6
Taxa de Acerto (Hit Rate): 62.50%

--- Análise de Efetividade vs. Comprimento da Sequência ---
Taxa de acerto do alvo 1.84x em cada etapa da sequência:


,low_streak,total_apostas,acertos,taxa_de_acerto_%
0,8,16,10,62.500000
1,9,6,3,50.000000
2,10,4,1,25.000000
3,11,3,1,33.333333
4,12,2,0,0.000000
5,13,2,2,100.000000


In [4]:
# Célula de Análise 3: Existe um Padrão Prévio para "Super-Streaks"?

print(f"Analisando padrões prévios de 'Super-Streaks' (sequências >= {SUPER_STREAK} baixos)\n")

# --- 1. Isolar os eventos de Fim de Sequência ---
# Pegamos apenas as linhas que marcam o fim de uma sequência de baixos
all_streaks = df[df['streak_end'] == True].copy()

# --- 2. Criar o Alvo (Target) ---
# Marcamos '1' se foi uma "Super-Streak", '0' se foi normal
all_streaks['is_super_streak'] = (all_streaks['streak_length'] >= SUPER_STREAK).astype(int)

# --- 3. Criar "Features Prévias" ---
# Vamos analisar as 20 rodadas ANTES do início de cada sequência
# (Vamos usar as features V2 que já testamos, pois são mais poderosas)

# Adiciona features de média e volatilidade ANTES da sequência começar
# (Usamos .shift(1) no df original)
df_features_v2 = df.copy()
for window_size in [5, 10, 20]:
    rolling_series = df_features_v2['multiplicador'].shift(1).rolling(window=window_size)
    df_features_v2[f'rolling_mean_{window_size}'] = rolling_series.mean()
    df_features_v2[f'rolling_std_{window_size}'] = rolling_series.std()
    
# Mapeia as features do df principal para o nosso df de sequências
# (Pega o valor da feature no momento em que a sequência terminou)
features_to_check = ['rolling_mean_5', 'rolling_mean_10', 'rolling_mean_20',
                     'rolling_std_5', 'rolling_std_10', 'rolling_std_20']
all_streaks[features_to_check] = df_features_v2.loc[all_streaks.index, features_to_check]

# Limpa qualquer NaN
all_streaks = all_streaks.dropna()

# --- 4. A Análise Estatística ---
# Agora, comparamos as features médias de sequências normais vs. Super-Streaks
print("--- Comparação Estatística (Média das Features Prévias) ---")
comparison = all_streaks.groupby('is_super_streak')[features_to_check].mean().T

# Renomeia colunas
comparison = comparison.rename(columns={0: 'Sequência Normal', 1: 'Super-Streak (12+)'})

# Calcula a diferença
comparison['Diferenca_%'] = ((comparison['Super-Streak (12+)'] - comparison['Sequência Normal']) / comparison['Sequência Normal']) * 100

display(comparison)

Analisando padrões prévios de 'Super-Streaks' (sequências >= 12 baixos)

--- Comparação Estatística (Média das Features Prévias) ---


is_super_streak,Sequência Normal,Super-Streak (12+),Diferenca_%
rolling_mean_5,5.633149,1.427000,-74.667812
rolling_mean_10,5.625106,1.470000,-73.867160
rolling_mean_20,5.564289,1.930250,-65.310034
rolling_std_5,7.395361,0.350595,-95.259264
rolling_std_10,9.270427,0.324307,-96.501708
rolling_std_20,11.130996,1.317772,-88.161239


In [5]:
# Célula de Análise 4: Preparando dados para prever Super-Streaks

print("Iniciando preparação de dados para o modelo 'Previsor de Super-Streak'...")

# --- 1. Recalcular/Garantir Features V2 (17 features) ---
# (Usamos as features mais poderosas que descobrimos)
TARGET_MULTIPLIER = 1.99 # Usando seu limite exato
df_features_v2 = df.copy()

for i in range(1, 11): # Lags 1-10
    df_features_v2[f'lag_{i}'] = df_features_v2['multiplicador'].shift(i)

for window_size in [5, 10, 20]: # Janelas 5, 10, 20
    rolling_series = df_features_v2['multiplicador'].shift(1).rolling(window=window_size)
    df_features_v2[f'rolling_mean_{window_size}'] = rolling_series.mean()
    df_features_v2[f'rolling_std_{window_size}'] = rolling_series.std()

is_low = df_features_v2['multiplicador'].shift(1) < TARGET_MULTIPLIER
cumulative_lows = is_low.astype(int).cumsum()
streak_breaks = cumulative_lows.where(~is_low)
filled_breaks = streak_breaks.ffill().fillna(0)
df_features_v2['low_streak'] = (cumulative_lows - filled_breaks).astype(int)

features_list_v2 = [col for col in df_features_v2.columns if col.startswith('lag_') or col.startswith('rolling_') or col == 'low_streak']

# --- 2. Criar o Alvo (y) ---
# Identificar o comprimento final de cada sequência
df_temp = df.copy()
df_temp['is_low'] = (df_temp['multiplicador'] < TARGET_MULTIPLIER)
df_temp['streak_id'] = df_temp['is_low'].diff().astype(float).fillna(0.0).cumsum()
streak_lengths = df_temp[df_temp['is_low'] == True].groupby('streak_id').size()
df_temp['final_streak_length'] = df_temp['streak_id'].map(streak_lengths)
# Preenche o comprimento para todas as linhas daquela sequência
df_temp['final_streak_length'] = df_temp['final_streak_length'].bfill(limit=100).fillna(0) 

# --- 3. Juntar Alvo (y) e Features (X) ---
# O alvo é: esta rodada ATUAL faz parte de uma futura Super-Streak?
df_features_v2['y_target_is_super_streak'] = (df_temp['final_streak_length'] >= SUPER_STREAK).astype(int)

# 4. Limpar NaNs (das primeiras 20 linhas que não têm features V2)
df_model_data = df_features_v2.dropna()

print(f"\nTotal de amostras de dados (com features V2): {len(df_model_data)}")
print(f"Total de amostras que pertencem a uma Super-Streak (12+): {df_model_data['y_target_is_super_streak'].sum()}")

# 5. Separar X e y
X_super = df_model_data[features_list_v2]
y_super = df_model_data['y_target_is_super_streak']

Iniciando preparação de dados para o modelo 'Previsor de Super-Streak'...

Total de amostras de dados (com features V2): 5891
Total de amostras que pertencem a uma Super-Streak (12+): 28


In [6]:
# Célula de Análise 5: Treinar um Modelo para Prever Super-Streaks

from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Garantir que temos dados suficientes e ambas as classes (0 e 1)
if len(y_super) < 50 or len(y_super.unique()) < 2:
    print("Não há dados suficientes ou variedade (Super-Streaks são muito raras) para treinar um modelo.")
else:
    print("Iniciando Validação Cruzada (TimeSeriesSplit) para prever Super-Streaks...")
    
    # Vamos usar 3 splits, pois os eventos de Super-Streak (1) são raros
    tscv_super = TimeSeriesSplit(n_splits=3) 
    # Usamos 'class_weight="balanced"' para dar mais importância aos eventos raros (as Super-Streaks)
    model_super = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
    scaler_super = StandardScaler()

    accuracies_super = []
    
    print("\n--- Relatórios de Validação (Foco na Classe '1' = Super-Streak) ---")
    
    for split_count, (train_index, test_index) in enumerate(tscv_super.split(X_super), 1):
        X_train, X_test = X_super.iloc[train_index], X_super.iloc[test_index]
        y_train, y_test = y_super.iloc[train_index], y_super.iloc[test_index]

        # Garantir que o split de teste tenha eventos de Super-Streak (classe 1)
        if y_test.sum() == 0:
            print(f"\n--- Split {split_count}/3 (Ignorado) ---")
            print("Nenhum evento de Super-Streak neste split de teste.")
            continue
            
        X_train_scaled = scaler_super.fit_transform(X_train)
        X_test_scaled = scaler_super.transform(X_test)
        
        model_super.fit(X_train_scaled, y_train)
        y_pred = model_super.predict(X_test_scaled)
        
        acc = accuracy_score(y_test, y_pred)
        accuracies_super.append(acc)
        print(f"\n--- Split {split_count}/3 (Acurácia: {acc:.3%}) ---")
        # Imprimir o relatório completo é mais importante aqui
        # 'precision' (classe 1): Das vezes que ele PREVIU uma Super-Streak, quantas ele acertou?
        # 'recall' (classe 1): De todas as Super-Streaks que ACONTECERAM, quantas ele previu?
        print(classification_report(y_test, y_pred, zero_division=0))

    print("-" * 40)
    print(f"Acurácia Média (Modelo Super-Streak): {np.mean(accuracies_super):.3%}")
    
    print("\n--- Treinando Modelo Final (Super-Streak) e Vendo Features Importantes ---")
    # Treinar no dataset completo (X_super, y_super)
    X_super_scaled = scaler_super.fit_transform(X_super)
    model_super.fit(X_super_scaled, y_super)
    
    # Ver as features mais importantes
    feature_importances = pd.Series(model_super.feature_importances_, index=X_super.columns)
    print("Features mais importantes para PREVER uma Super-Streak (Fase Arrecadatória):")
    display(feature_importances.nlargest(10))

Iniciando Validação Cruzada (TimeSeriesSplit) para prever Super-Streaks...

--- Relatórios de Validação (Foco na Classe '1' = Super-Streak) ---

--- Split 1/3 (Acurácia: 98.981%) ---
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1458
           1       0.00      0.00      0.00        14

    accuracy                           0.99      1472
   macro avg       0.50      0.50      0.50      1472
weighted avg       0.98      0.99      0.99      1472


--- Split 2/3 (Ignorado) ---
Nenhum evento de Super-Streak neste split de teste.

--- Split 3/3 (Ignorado) ---
Nenhum evento de Super-Streak neste split de teste.
----------------------------------------
Acurácia Média (Modelo Super-Streak): 98.981%

--- Treinando Modelo Final (Super-Streak) e Vendo Features Importantes ---
Features mais importantes para PREVER uma Super-Streak (Fase Arrecadatória):


rolling_std_5      0.163217
rolling_mean_10    0.153611
rolling_std_10     0.111297
rolling_mean_5     0.100302
low_streak         0.082427
rolling_std_20     0.065659
rolling_mean_20    0.050352
lag_1              0.038848
lag_10             0.038609
lag_7              0.029753
dtype: float64